In [140]:
import numpy as np
from fractions import Fraction
from sympy import symbols, And, Not, simplify, solve, reduce_inequalities
from sympy.core.assumptions import assumptions
from sympy import linprog

In [117]:
OUTCOME_PROB = Fraction(1,6)

In [162]:
x, y, z = symbols('x y z')
#assumptions(x)

In [163]:
V = [x>0, x<y, y<z]

def no_loser(a,b,c):
    return a>0 & b>0 & c>0

In [198]:
# The assumptions framework in sympy does not work so well with linear inequalities
# We use our own LP framework to detect:
# 1. constraints that violate V
# 2. inferences (e.g. 0<x<y and 3*x < y => 2*x < y)
from sympy.solvers.simplex import lpmax
from sympy.core import *

EPS = 1e-65
def to_nonstrict_ineq(ineq):
    if isinstance(ineq, Gt):
        # a>b => a >= b+EPS
        return Ge(ineq.lhs, ineq.rhs + EPS)
    elif isinstance(ineq, Lt):
        # a<b => a <= b-EPS
        return Le(ineq.lhs, ineq.rhs - EPS)
    else:
        return ineq  # already non-strict

In [199]:
from sympy.solvers.simplex import InfeasibleLPError

def solve_lp(e, ineqs, f=lpmax):
    # convert all constraints to non-strict inequalities
#     print(ineqs)
#     print(constraints)
    return f(e, [to_nonstrict_ineq(c) for c in ineqs])

def feasible(ineq, assumptions):
    try:
        result = solve_lp(0, assumptions + [ineq])
    except InfeasibleLPError:
        return False
    return result

def redundant(ineq, assumptions):
    # if Assumptions => Ineq (<=> not A or I) is valid, then (A and not I) should be unsatisfiable
    return feasible(Not(ineq), assumptions) == False

In [200]:
def getParentStates(a,b,c):
    return [(2*a,b-a,c), (2*a,b,c-a),
            (a-b,2*b,c), (a,2*b,c-b),
            (a-c,b,2*c), (a,b-c,2*c)]

In [123]:
getParentStates(y, 2*x, z-x)

[(2*y, 2*x - y, -x + z),
 (2*y, 2*x, -x - y + z),
 (-2*x + y, 4*x, -x + z),
 (y, 4*x, -3*x + z),
 (x + y - z, 2*x, -2*x + 2*z),
 (y, 3*x - z, -2*x + 2*z)]

In [125]:
import collections
import functools

class memoized(object):
    '''Decorator. Caches a function's return value each time it is called.
    If called later with the same arguments, the cached value is returned
    (not reevaluated).
    '''
    def __init__(self, func):
        self.func = func
        self.cache = {}
    
    def __call__(self, *args):
        if not isinstance(args, collections.Hashable):
            # uncacheable. a list, for instance.
            # better to not cache than blow up.
            return self.func(*args)
        if args in self.cache:
            return self.cache[args]
        else:
            value = self.func(*args)
            self.cache[args] = value
            return value
    
    def __repr__(self):
        '''Return the function's docstring.'''
        return self.func.__doc__
    
    def __get__(self, obj, objtype):
        '''Support instance methods.'''
        return functools.partial(self.__call__, obj)

In [213]:
def reduce(ineqs, assumptions):
    # converts the inferred inequalities under `assumptions` to 0s or 1s
    reduced = []
    const = 0
    for ineq in ineqs:
        if not feasible(ineq, assumptions):
            # not feasible
            continue
        elif redundant(ineq, assumptions):
            # definitely true under assumptions
            const += 1
        else:
            reduced.append(ineq)
    return const, reduced

In [222]:
@memoized
def _h(n,a,b,c):
    # constant coef = (1/6)^n
    # so we only keep track of the indicators to be summed up
    if n == 1:
        return [simplify(a<=b), simplify(a<=c)]
    # list of inequalities representing the region Rn
    return list(set(sum([_h(n-1, *s) for s in getParentStates(a,b,c)], [])))

# @memoized
# def h(n,a,b,c):
#     return reduce(_h(n,a,b,c), V)

class h():
    def __init__(self,n,a,b,c):
        self.coef = pow(OUTCOME_PROB, n)
        self.const = 0
        self.inds = _h(n,a,b,c)
        self.const, self.inds = self.reduce(r=V)
    
    def reduce(self, r=V):
        const = 0
        reduced = []
        # converts the inferred inequalities under `assumptions` to 0s or 1s
        for ind in self.inds:
            if not feasible(ind, r):
                # not feasible
                continue
            elif redundant(ind, r):
                # definitely true within the given region
                const += 1
            else:
                reduced.append(ind)
        return const, reduced
        
class dh():
    def __init__(self,n,a,b,c):
        self.pos = h(n,a,b,c)
        self.neg = h(n,b,a,c)
    
    def lb(self, r):
        const = self.pos.const - self.neg.const
        pos_c, pos_inds = self.pos.reduce(r=r)
        neg_c, neg_inds = self.neg.reduce(r=r)
        # detect relationship between positive and negative indicators
        # want lower bound - maximize neg inds & minimize pos inds
        # SIMPLIFICATION: assume all uncertain pos inds are false & neg inds are true
        return (const + (pos_c - neg_c) - len(neg_inds)) * self.pos.coef

def f(n, r):
    @memoized
    def sum_dh(n):
        if n == 1:
            return dh(1,x,y,z).lb(r)
        return dh(n,x,y,z).lb(r) + sum_dh(n-1)
    return sum_dh(n)

In [223]:
f(3, [3*x>y, z>3*y] + V)

Fraction(31, 216)

In [229]:
def thresh(n):
    return pow(Fraction(1,2), n) * Fraction(4,5)

In [231]:
def run(n):
    d = dh(n,x,y,z)
    for r in d.pos.inds:
        for s in d.neg.inds:
            fval = f(n, V + [r,s])
            print([r,s], fval)
            if fval <= thresh(n):
                print("Failed!")
                return run(n+1)
    return n

In [ ]:
run(2)

[2*x <= y, y <= 3*x] 5/36
Failed!
[z >= 2*x, 5*x >= 3*y] 2/27
Failed!
[3*x <= -2*y + 2*z, 5*x >= 3*y + z] 193/648
[3*x <= -2*y + 2*z, 3*x <= -y + 2*z] 35/432
[3*x <= -2*y + 2*z, 5*x >= 5*y - z] 107/1296
[3*x <= -2*y + 2*z, z >= 15*y] 37/216
[3*x <= -2*y + 2*z, 3*x <= -5*y + 3*z] 55/648
[3*x <= -2*y + 2*z, 2*x >= 4*y - z] 1/12
[3*x <= -2*y + 2*z, 2*x >= 6*y - z] 61/432
[3*x <= -2*y + 2*z, 6*x >= 2*y - z] 35/432
